In [1]:
#Import Modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pyproj
from sqlalchemy import create_engine
from bokeh.plotting import figure, output_file, show
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.tile_providers import CARTODBPOSITRON, WIKIMEDIA
from bokeh.palettes import Spectral6
from bokeh.io import curdoc, export_svgs, export_png
from bokeh.models import(ColumnDataSource, Grid, LinearAxis, Plot, Quad ,Range1d, HoverTool, Panel,
Tabs, Legend, LegendItem, CDSView, BooleanFilter, CustomJSFilter)

# Files to Load
csvfile = "C:/Users/arodr/OneDrive/Desktop/CTA Transit/CTA_Ridership.csv"
csvfile2 = "C:/Users/arodr/OneDrive/Desktop/CTA Transit/CTA_Train_Stations.csv"
combined_csvfile = "C:/Users/arodr/OneDrive/Desktop/CTA Transit/merged_tables.csv"
# Create dataframe for loaded files
CTA_Ridership_df = pd.read_csv(csvfile, sep=',')
CTA_Train_Stations_df = pd.read_csv(csvfile2, sep=',')
CTA_Merged_Tables_df = pd.read_csv(combined_csvfile, sep=',')
CTA_Train_Stations_df.head

<bound method NDFrame.head of      STOP_ID                        STOP_NAME          STATION_NAME  MAP_ID  \
0      30162         18th (54th/Cermak-bound)                  18th   40830   
1      30161                18th (Loop-bound)                  18th   40830   
2      30022         35th/Archer (Loop-bound)           35th/Archer   40120   
3      30023       35th/Archer (Midway-bound)           35th/Archer   40120   
4      30214  35-Bronzeville-IIT (63rd-bound)  35th-Bronzeville-IIT   41120   
..       ...                              ...                   ...     ...   
295    30105            Wilson (Howard-bound)                Wilson   40540   
296    30239           Addison (O'Hare-bound)               Addison   41240   
297    30106              Wilson (95th-bound)                Wilson   40540   
298    30383   Washington/Wabash (Outer Loop)     Washington/Wabash   41700   
299    30385              Wilson (Loop-bound)                Wilson   40540   

     A_D_A    RED   B

In [2]:
new_CTA_Ridership_data_df = CTA_Ridership_df [['station_id', 'station_name', 'rides', 'date']].copy()
new_CTA_Ridership_data_df.head(15)

,station_id,station_name,rides,date
0,41280,Jefferson Park,"6,104",12/22/2017
1,41000,Cermak-Chinatown,"3,636",12/18/2017
2,40280,Central-Lake,"1,270",12/2/2017
3,40140,Dempster-Skokie,"1,759",12/19/2017
4,40690,Dempster,499,12/3/2017
5,41660,Lake/State,"8,615",12/30/2017
6,40180,Oak Park-Forest Park,442,12/17/2017
7,40250,Kedzie-Homan-Forest Park,"1,353",12/2/2017
8,40120,35th/Archer,"3,353",12/7/2017
9,41420,Addison-North Main,"6,034",12/19/2017


In [3]:
new_CTA_Ridership_data_df.date = pd.to_datetime(new_CTA_Ridership_data_df.date)
date_sort = new_CTA_Ridership_data_df.sort_values(by = 'date', ascending = True)
new_CTA_Ridership_data_df.date = pd.to_datetime(new_CTA_Ridership_data_df.date)

start_date = "2019-07-01"
end_date = "2021-2-6"

after_start_date = new_CTA_Ridership_data_df["date"] >= start_date
before_end_date = new_CTA_Ridership_data_df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = new_CTA_Ridership_data_df.loc[between_two_dates]
filtered_dates.head(15)

,station_id,station_name,rides,date
52598,40010,Austin-Forest Park,"1,870",2019-07-01
52599,40010,Austin-Forest Park,"1,887",2019-07-02
52600,40010,Austin-Forest Park,"1,815",2019-07-03
52601,40010,Austin-Forest Park,770,2019-07-04
52602,40010,Austin-Forest Park,"1,244",2019-07-05
52603,40010,Austin-Forest Park,783,2019-07-06
52604,40010,Austin-Forest Park,657,2019-07-07
52605,40010,Austin-Forest Park,"1,844",2019-07-08
52606,40010,Austin-Forest Park,"1,980",2019-07-09
52607,40010,Austin-Forest Park,"1,916",2019-07-10


In [4]:
new_CTA_Train_Stations_df = CTA_Train_Stations_df [["MAP_ID", "STATION_NAME", "A_D_A", "RED", "BLUE", "GREEN", "BROWN", "PURPLE", "PURPLE_EXPRESS", "YELLOW", "PINK", "ORANGE", "Lat_Lon"]].copy()
new_CTA_Train_Stations_df.head(15)

,MAP_ID,STATION_NAME,A_D_A,RED,BLUE,GREEN,BROWN,PURPLE,PURPLE_EXPRESS,YELLOW,PINK,ORANGE,Lat_Lon
0,40830,18th,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,40830,18th,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,40120,35th/Archer,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,40120,35th/Archer,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,41120,35th-Bronzeville-IIT,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"
5,41120,35th-Bronzeville-IIT,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"
6,41270,43rd,True,False,False,True,False,False,False,False,False,False,"(41.816462, -87.619021)"
7,41270,43rd,True,False,False,True,False,False,False,False,False,False,"(41.816462, -87.619021)"
8,40130,51st,True,False,False,True,False,False,False,False,False,False,"(41.80209, -87.618487)"
9,40130,51st,True,False,False,True,False,False,False,False,False,False,"(41.80209, -87.618487)"


In [5]:
import psycopg2
psycopg2.connect("dbname=postgres user=postgres host=localhost password=postgres port=5432")

<connection object at 0x0000017D8B888480; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5432', closed: 0>

In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/Train_Stations_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

['cta_stations', 'ridership_data']

In [8]:
filtered_dates.to_sql(name='ridership_data', con=engine, if_exists='append', index=False)

In [9]:
new_CTA_Train_Stations_df.to_sql(name='cta_stations', con=engine, if_exists='append', index=False)

In [10]:
pd.read_sql_query('select * from ridership_data', con=engine).head()

,station_id,station_name,rides,date
0,40010,Austin-Forest Park,"1,870",2019-07-01
1,40010,Austin-Forest Park,"1,887",2019-07-02
2,40010,Austin-Forest Park,"1,815",2019-07-03
3,40010,Austin-Forest Park,770,2019-07-04
4,40010,Austin-Forest Park,"1,244",2019-07-05


In [11]:
pd.read_sql_query('select * from cta_stations', con=engine).head()

,MAP_ID,STATION_NAME,A_D_A,RED,BLUE,GREEN,BROWN,PURPLE,PURPLE_EXPRESS,YELLOW,PINK,ORANGE,Lat_Lon
0,40830,18th,true,false,false,false,false,false,false,false,true,false,"(41.857908, -87.669147)"
1,40830,18th,true,false,false,false,false,false,false,false,true,false,"(41.857908, -87.669147)"
2,40120,35th/Archer,true,false,false,false,false,false,false,false,false,true,"(41.829353, -87.680622)"
3,40120,35th/Archer,true,false,false,false,false,false,false,false,false,true,"(41.829353, -87.680622)"
4,41120,35th-Bronzeville-IIT,true,false,false,true,false,false,false,false,false,false,"(41.831677, -87.625826)"


In [12]:
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [13]:
CTA_Merged_Tables_df['coords_x'] = CTA_Merged_Tables_df['location'].apply(lambda x: merc(x)[0])
CTA_Merged_Tables_df['coords_y'] = CTA_Merged_Tables_df['location'].apply(lambda x: merc(x)[1])

In [126]:
#remove duplicate rides --no "to" AND "from" rides, only one 
CTA_Merged_Tables_df.drop_duplicates(subset='station_id', keep="last", inplace=True)

In [127]:
from bokeh.tile_providers import get_provider, Vendors 
tile_provider = get_provider(Vendors.WIKIMEDIA)
from bokeh.plotting import figure, show, output_notebook

p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator",
           tools=['pan', 'wheel_zoom','save'])
p.add_tile(tile_provider)
source = ColumnDataSource(data=dict(
                        x=list(CTA_Merged_Tables_df['coords_x']), 
                        y=list(CTA_Merged_Tables_df['coords_y'])))

p.circle(x = CTA_Merged_Tables_df['coords_x'],
         y = CTA_Merged_Tables_df['coords_y'])

output_notebook()
show(p)

Loading BokehJS ...

In [158]:
Station_Group = CTA_Merged_Tables_df.groupby(['station_name',])
Station_Totals = pd.DataFrame(Station_Group['rides'].sum())
Station_Totals.head(30)

,rides
station_name,
18th,362
35-Bronzeville-IIT,230
35th/Archer,315
43rd,128
47th-Dan Ryan,445
47th-South Elevated,210
51st,210
54th/Cermak,416
63rd-Dan Ryan,630


In [140]:
CTA_Merged_Tables_df['circle_sizes'] = CTA_Merged_Tables_df['rides'] / 35

In [141]:
from bokeh.tile_providers import get_provider, Vendors 
tile_provider = get_provider(Vendors.WIKIMEDIA)
from bokeh.plotting import figure, show, output_notebook

# range bounds supplied in web mercator coordinates
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
p.circle(x=CTA_Merged_Tables_df['coords_x'],
         y=CTA_Merged_Tables_df['coords_y'], 
         size=CTA_Merged_Tables_df['circle_sizes'],
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05)
    
output_notebook()
show(p)

Loading BokehJS ...

In [142]:
from bokeh.tile_providers import get_provider, Vendors 
tile_provider = get_provider(Vendors.WIKIMEDIA)
from bokeh.plotting import figure, show, output_notebook


source = ColumnDataSource(data=dict(
                        x=list(CTA_Merged_Tables_df['coords_x']), 
                        y=list(CTA_Merged_Tables_df['coords_y']),
                        Station_Totals=list(CTA_Merged_Tables_df['rides']),
                        sizes=list(CTA_Merged_Tables_df['circle_sizes']),
                        stationname=list(CTA_Merged_Tables_df['station_name'])))
hover = HoverTool(tooltips=[
    ("station", "@stationname"),
    ("ridership","@Station_Totals")
    ])


# range bounds supplied in web mercator coordinates
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
           tools=[hover, 'wheel_zoom','save'])
p.add_tile(tile_provider)
p.circle(x='x',
         y='y', 
         source=source,
         size='sizes',
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05)
    
output_notebook()
show(p)

Loading BokehJS ...